In [21]:
import sys

sys.path.insert(0, '..')

In [17]:
from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
from core.dataset_multimodal import collate_fn, ASASSNVarStarDataset
from functools import partial
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import random
import numpy as np
import json
import os

In [3]:
random.seed(42)

In [4]:
datapath = Path('../data/asaasn')
ds = ASASSNVarStarDataset(datapath, mode='train', verbose=True, only_periodic=True,
                          merge_type='inner', recalc_period=False, prime=True, use_bands=['v', 'g'],
                          only_sources_with_spectra=False)

Opening v data files... Found 687695 sources. done.
Opening g data files... Found 378861 sources. done.
Opening spectra csv... done.
Removing duplicates for v band... Left with 675811. done.
Removing duplicates for g band... Left with 378823. done.
Merging bands... done. Now 268730 sources.
Removed non-periodic sources. Now 219897 sources.
Total: 219897, Train: 175917, Val: 21989, Test: 21991
Priming tarballs by doing initial scan... done.


In [8]:
total_flux_v = ds[0]['lcs'][0][0][:, 1]
total_flux_g = ds[0]['lcs'][0][1][:, 1]

In [6]:
indx = random.sample(range(1, len(ds)), 999)

In [9]:
for i in tqdm(indx):
    flux_v = ds[i]['lcs'][0][0][:, 1]
    flux_g = ds[i]['lcs'][0][1][:, 1]

    total_flux_v = np.concatenate((total_flux_v, flux_v))
    total_flux_g = np.concatenate((total_flux_g, flux_g))

100%|███████████████████████████████████████████████████████████████| 999/999 [01:26<00:00, 11.56it/s]


In [10]:
len(total_flux_v), len(total_flux_g)

(946850, 519599)

In [11]:
total_flux_v.mean(), total_flux_v.std()

(19.19374651317585, 18.637131647187786)

In [12]:
total_flux_g.mean(), total_flux_g.std()

(9.255897176476472, 13.29101329113628)

In [14]:
res = {
    'v': {'mean': total_flux_v.mean(), 'std': total_flux_v.std()},
    'g': {'mean': total_flux_g.mean(), 'std': total_flux_g.std()}
}

In [18]:
with open(os.path.join(datapath, 'scales.json'), 'w') as file:
    json.dump(res, file)

In [20]:
ds.data_root / 'scales.json'

PosixPath('../data/asaasn/scales.json')